## Example: Single Index Models and MinVar Portfolio Allocation
Fill me in

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Examples-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Installed XZ_jll ── v5.4.5+0
   Installed MbedTLS ─ v1.1.8
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
    Updating `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  [739be429] ↑ MbedTLS v1.1.7 ⇒ v1.1.8
  [ffd25f8a] ↑ XZ_jll v5.4.4+0 ⇒ v5.4.5+0

## Prerequisites: Load and clean the historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `11-03-2023`, along with data for a few exchange traded funds and volatility products during that time. We load the `orignal_dataset` by calling the `load(...)` method exported by the [JLD2.jl](https://github.com/JuliaIO/JLD2.jl.git) package:

In [2]:
original_dataset = load(joinpath(_PATH_TO_DATA, "SP500-Daily-OHLC-1-3-2018-to-11-03-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of tradition days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

In [4]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it ib the `all_tickers` array

In [5]:
all_tickers = keys(dataset) |> collect |> sort;

Finally, let's set some constant values

In [6]:
risk_free_rate = 0.05;

### Compute the covariance and expected yield for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [7]:
all_firms_excess_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = risk_free_rate);

Next, we estimate the expected excess return for each firm from the `all_firms_excess_return_matrix` using the `mean(...)` function, which is exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We set the $N\times{1}$ vector of expected values, i.e., $\mathbb{E}(R_{1}),\dotsc,\mathbb{E}(R_{N})$ in the variable $\mu$:

In [8]:
μ = mean(all_firms_excess_return_matrix, dims=1) |> vec;

Finally, we estimate the annualized `covariance_matrix` from the `all_firms_return_matrix` using the `cov(...)` function, exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We store the $N\times{N}$ covariance matrix in the $\Sigma$ variable:

In [11]:
Σ = cov(all_firms_excess_return_matrix) |> x-> x*(1/252); # annualized, historical volatility

### Build single index model returns and covariance

In [13]:
sims = load(joinpath(_PATH_TO_DATA, "SIM-SP500-01-03-18-to-10-31-23.jld2")) |> x->x["sim"]

Dict{String, MySingleIndexModel} with 459 entries:
  "NI"   => MySingleIndexModel(-0.0667694, 0.650089, 0.05, Normal{Float64}(μ=4.…
  "EMR"  => MySingleIndexModel(-0.0420337, 1.23606, 0.05, Normal{Float64}(μ=8.6…
  "CTAS" => MySingleIndexModel(0.121342, 1.18426, 0.05, Normal{Float64}(μ=-3.75…
  "HSIC" => MySingleIndexModel(-0.0887318, 0.918814, 0.05, Normal{Float64}(μ=7.…
  "KIM"  => MySingleIndexModel(-0.0834768, 1.28034, 0.05, Normal{Float64}(μ=-1.…
  "PLD"  => MySingleIndexModel(0.00268562, 1.01582, 0.05, Normal{Float64}(μ=7.3…
  "IEX"  => MySingleIndexModel(-0.00997753, 0.962278, 0.05, Normal{Float64}(μ=3…
  "BAC"  => MySingleIndexModel(-0.105029, 1.31866, 0.05, Normal{Float64}(μ=-1.6…
  "CBOE" => MySingleIndexModel(-0.0157717, 0.594397, 0.05, Normal{Float64}(μ=-3…
  "EXR"  => MySingleIndexModel(-0.0361987, 0.709725, 0.05, Normal{Float64}(μ=1.…
  "NCLH" => MySingleIndexModel(-0.349225, 2.33008, 0.05, Normal{Float64}(μ=3.66…
  "CVS"  => MySingleIndexModel(-0.0803743, 0.78294, 0.05, 

In [19]:
index_SPY = findfirst(x->x=="SPY", all_tickers);
expected_return_SPY = μ[index_SPY];

### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.